In [6]:
import sys
sys.path.append("../")

In [1]:
import xarray as xr
xr.open_dataset('/mnt/bucket/stuarts_files/glofas/1999/CEMS_ECMWF_dis24_19990104_glofas_v2.1.nc')

<xarray.Dataset>
Dimensions:  (lat: 1500, lon: 3600, time: 1)
Coordinates:
  * time     (time) datetime64[ns] 1999-01-04
  * lon      (lon) float64 -179.9 -179.8 -179.8 -179.6 ... 179.8 179.9 180.0
  * lat      (lat) float64 89.95 89.85 89.75 89.65 ... -59.75 -59.85 -59.95
Data variables:
    dis24    (time, lat, lon) float32 ...
Attributes:
    CDI:                       Climate Data Interface version 1.9.6 (http://m...
    Conventions:               CF-1.6
    history:                   Thu Oct 10 12:28:52 2019: cdo -seldate,1999-01...
    cdo_openmp_thread_number:  8
    NCO:                       netCDF Operators version 4.7.8 (Homepage = htt...
    CDO:                       Climate Data Operators version 1.9.6 (http://m...

In [7]:
from functions.floodmodel_utils import get_basin_mask, get_river_mask, reshape_scalar_predictand, reshape_multiday_predictand
import xarray as xr
#Creating a Dask local cluster for parallel computing (making the computations later on much faster)
from dask.distributed import Client, LocalCluster


In [8]:

#HYPERPARAMETERS
days_intake_length = 90
forecast_day = 30


#LIBRARY IMPORTS
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from dask.distributed import Client, LocalCluster


#Connecting to a cluster to be able to run the code locally/on the cloud
#cluster = LocalCluster()  # n_workers=10, threads_per_worker=1,
client = Client(processes=False)


/root/miniconda3/envs/main/lib/python3.6/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [9]:
#This will tell you where you dashboard will be so you can visualize your model being run
print(client.scheduler_info()['services'])

{'dashboard': 34803}


In [ ]:
glofas_loaded = xr.open_mfdataset("/mnt/bucket/stuarts_files/glofas/*/*.nc", combine="by_coords")


In [ ]:
era5_loaded = xr.open_mfdataset("/mnt/bucket/stuarts_files/Elbe/reanalysis-era5-single-levels_convective_precipitation,land_sea_mask,large_scale_precipitation,runoff,slope_of_sub_gridscale_orography,soil_type,total_column_water_vapour,volumetric_soil_water_layer_1,volumetric_soil_water_layer_2_*.nc", combine="by_coords")

In [ ]:
glofas = glofas_loaded.copy()
era5 = era5_loaded.copy()
glofas = glofas.rename({'lon' : 'longitude'})
glofas = glofas.rename({'lat': 'latitude'})


In [ ]:
glofas_loaded

In [ ]:
elbe_basin_mask = get_basin_mask(glofas['dis24'].isel(time=0), 'Elbe')
elbe_river_mask = get_river_mask(glofas['dis24'].isel(time=0))


In [ ]:
glofas_masked = glofas.where(elbe_basin_mask, drop=True)
glofas_river_masked = glofas_masked.where(elbe_river_mask, drop=True)


In [ ]:
glofas_masked['dis24'].isel(time=0).plot()

In [ ]:
glofas_river_masked['dis24'].isel(time=0).plot()

In [ ]:
y_orig = glofas_masked['dis24']
#Making a copy because y will be transformed to represent the variation of discharge. The model will be predicting the variation of discharge, not the quantity of discharge itself
y = y_orig.copy()

#Era5 will be the predictor dataset


In [ ]:
y.to_netcdf("/mnt/bucket/stuarts_files/elbedischarge.nc")

In [17]:
era5 = era5.interp(latitude=glofas.latitude, longitude=glofas.longitude).where(elbe_basin_mask, drop=True)
X = era5


In [1]:
X.compute()

NameError: name 'X' is not defined

In [19]:
y

<xarray.DataArray 'dis24' (time: 7610, latitude: 54, longitude: 71)>
dask.array<where, shape=(7610, 54, 71), dtype=float32, chunksize=(1, 54, 71), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float64 9.25 9.35 9.45 9.55 ... 16.05 16.15 16.25
  * latitude   (latitude) float64 53.85 53.75 53.65 53.55 ... 48.75 48.65 48.55
  * time       (time) datetime64[ns] 1999-01-01 1999-01-02 ... 2019-11-01
Attributes:
    units:    m3/s

In [20]:
X_small = X.mean(['longitude', 'latitude'])
y_small = y.mean(['latitude','longitude'])

In [21]:
period_train = dict(time=slice(None, '1999-01-15'))


X_small, y_small = X_small.loc[period_train], y_small.loc[period_train]


In [22]:
X_small.compute()

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 190.19 GB -- Worker memory limit: 268.11 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 191.83 GB -- Worker memory limit: 268.11 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 193.78 GB -- Worker memory limit: 268.11 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 195.74 GB -- Worker memory limit: 268.11 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 197.70 GB -- Worker memory limit: 268.11 GB
distr

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 226.65 GB -- Worker memory limit: 268.11 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 226.94 GB -- Worker memory limit: 268.11 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 227.25 GB -- Worker memory limit: 268.11 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 227.51 GB -- Worker memory limit: 268.11 GB


KeyboardInterrupt: 

In [ ]:
y_small.compute()

In [ ]:
Xda, yda = reshape_scalar_predictand(X_small, y_small)

In [ ]:
Xda

In [ ]:
yda

In [ ]:
concatenated = xr.concat([Xda, Xda], 'points')

In [ ]:
concatenated

In [ ]:
from tqdm import tqdm
for i in tqdm(range(0, len(X.latitude), 10)):

    for j in range(0, len(X.longitude), 10):
        
        if i==0 and j==0:
            Xda, yda = reshape_scalar_predictand(X.isel(latitude=i, longitude=j), y.isel(latitude=i, longitude=j))
            
        else:
            XdaNew, ydaNew = reshape_scalar_predictand(X.isel(latitude=i, longitude=j), y.isel(latitude=i, longitude=j))

            Xda = xr.concat([Xda, XdaNew], 'points')
            yda = xr.concat([yda, ydaNew], 'points')
            



In [ ]:
Xda.to_netcdf('/mnt/bucket/stuarts_files/elbe.nc')

In [ ]:
yda.to_netcdf('mnt/bucket/starts_files/elbeglofas.nc')

In [ ]:
period_train = dict(time=slice(None, '2005'))
period_valid = dict(time=slice('2006', '2011'))
period_test = dict(time=slice('2012', '2016'))


X_train, y_train = Xda.loc[period_train], yda.loc[period_train]
X_valid, y_valid = Xda.loc[period_valid], yda.loc[period_valid]
X_test, y_test = Xda.loc[period_test], yda.loc[period_test]


In [45]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
X_train_scaled = sc.fit_transform(X_train)
y_train_scaled = y_train
#I'm hesitating whether I should apply standard scaling on the output
# sc2 = MinMaxScaler(feature_range=(0,1))
# y_train_scaled = sc2.fit_transform(y_train.values.reshape(-1,1))
X_train = []
y_train_array = []

coordinate_array = []

#tqdm, itertools.product.
#Iterating through each feature, shifting the time for each feature, and appending the time-shifted feature array to X_train for a total of 16 times
for n in range(len(X_train.data_vars)):

    feature_array = []
    #For every latitude coordinate:
    for j in range(len(X_train.latitude)):

        for k in range(len(X_train.longitude)):

            for l in range(60, len(X_train_scaled)):

                feature_array.append(X_train.isel(latitude=j, longitude=k)[l - days_intake_length:l, n])




            coordinate_array.append(feature_array)
            feature_array = []




    X_train.append(coordinate_array)



<xarray.Dataset>
Dimensions:    (latitude: 54, longitude: 71, time: 180367)
Coordinates:
  * latitude   (latitude) float64 53.85 53.75 53.65 53.55 ... 48.75 48.65 48.55
  * longitude  (longitude) float64 9.25 9.35 9.45 9.55 ... 16.05 16.15 16.25
  * time       (time) datetime64[ns] 1999-01-01 ... 2019-11-29T06:00:00
Data variables:
    cp         (time, latitude, longitude) float32 dask.array<chunksize=(744, 54, 71), meta=np.ndarray>
    lsm        (time, latitude, longitude) float32 dask.array<chunksize=(744, 54, 71), meta=np.ndarray>
    lsp        (time, latitude, longitude) float32 dask.array<chunksize=(744, 54, 71), meta=np.ndarray>
    ro         (time, latitude, longitude) float32 dask.array<chunksize=(744, 54, 71), meta=np.ndarray>
    slor       (time, latitude, longitude) float32 dask.array<chunksize=(744, 54, 71), meta=np.ndarray>
    slt        (time, latitude, longitude) float32 dask.array<chunksize=(744, 54, 71), meta=np.ndarray>
    tcwv       (time, latitude, longitude) float32 dask.array<chunksize=(744, 54, 71), meta=np.ndarray>
    swvl1      (time, latitude, longitude) float32 dask.array<chunksize=(744, 54, 71), meta=np.ndarray>
    swvl2      (time, latitude, longitude) float32 dask.array<chunksize=(744, 54, 71), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2019-12-02 16:58:32 GMT by grib_to_netcdf-2.14.0: /opt/ecmw...